In [1]:
%matplotlib inline
import matplotlib as mpl
mpl.style.use('/Users/nik/Dropbox/Notebooks/thesisrc.py')
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import glob
import os
import pandas as pd
import lvdatatools as lvdt
import re
import math

Going to try to find a noisy section of a quantum dot and exract some statistics about the random telegraph switching. The goal here is to plot the distribution of switching times as a fucntion of gate voltage over a noisy region of a coulomb diamond plot.

In [2]:
pkl_dir = '/Volumes/JUPITER/Data/fm_dots/SCF96/'
fig_dir = '/Volumes/JUPITER/Data/fm_dots/SCF96/figures/'
og_dir = '/Volumes/JUPITER/Data/SCF96/q3/'
os.listdir(pkl_dir)

OSError: [Errno 2] No such file or directory: '/Volumes/JUPITER/Data/fm_dots/SCF96/'

In [ ]:
cond = pd.read_pickle(os.path.join(pkl_dir, '4K_vi-gate_16-17_23_a_2T_cond.pkl'))

fig, ax = plt.subplots(figsize=(18,10))
limits = lvdt.df_extent(cond)

im = ax.imshow(cond, extent = limits, 
               cmap = plt.cm.seismic, vmin = -0.075, vmax = 0.125, 
               origin = 'lower', aspect = 'auto', interpolation = 'nearest')
ax.set_xlabel(r'$V_G (V)$')
ax.set_ylabel(r'$V_B (mV)$')
#ax.set_title(r'conductance ($\frac{e^2}{h}$)')
ax.set_xlim(15, 30)
cb = plt.colorbar(im)
cb.set_label(r'$G$')

In [ ]:
gate = cond.columns.values
filelist = [os.path.join(og_dir, 'vi-gate_16-17_23_a_2T_{0:.0f}.bin'.format(g*1000)) for g in gate]
print np.all([os.path.isfile(f) for f in filelist]) # if this is false there is going to be trouble

Great. Now I have the filelist for the dataset I want to work with. Time to start looking at some time series.

In [ ]:
ind = np.where(cond.columns.values==23)
df = lvdt.get_data_2d(filelist[ind[0]])
df = df.groupby(df['Bias']).mean()
fig, ax = plt.subplots()
ax.plot(df.index*1e3, df['Iavg']*1e9)

In [ ]:
ind = np.where(cond.columns.values==23)
df = lvdt.get_data_2d(filelist[ind[0]])

dt = df['dt'].mean()
N = len(df.loc[0,'i0':])
t = np.arange(0,N)*dt

M = 10
fig, axs = plt.subplots(M, 1,sharex = True, sharey=False, 
                        figsize=(18,M*1.5))
axs = axs.flat

sample = np.linspace(0,len(df['Bias'])-1,M, dtype='int')

for i, ax in enumerate(axs):
    ts = df.loc[sample[i],'i0':].values*1e9
    bias = df['Bias'][sample[i]]*1e3
    ax.plot(t, ts, label = bias)
    #ax.plot(np.diff(ts), label = bias)
    #ax.set_xlabel('Gate (V)')
    #ax.set_ylabel('I (nA)')
    ax.legend()
ax.set_xlabel('time (s)')

In [ ]:
ind = np.where(cond.columns.values==23)
df = lvdt.get_data_2d(filelist[ind[0]])

dt = df['dt'].mean()
N = len(df.loc[0,'i0':])
t = np.arange(0,N)*dt

M = 10
fig, axs = plt.subplots(M, 1,sharex = True, sharey=False, 
                        figsize=(18,M*1.5))
axs = axs.flat

sample = np.linspace(0,len(df['Bias'])-1,M, dtype='int')

for i, ax in enumerate(axs):
    ts = df.loc[sample[i],'i0':].values*1e9
    bias = df['Bias'][sample[i]]*1e3
    #ax.plot(t, ts, label = bias)
    ax.plot(abs(np.diff(ts)), 'r', label = bias)
    #ax.set_xlabel('Gate (V)')
    #ax.set_ylabel('I (nA)')
    ax.set_ylim(-0.1, None)
    ax.legend()
ax.set_xlabel('time (s)')

That's a good illustration of what I'm working with here. What I need is some way to scale the measured currents in the time series by the 'expected' value so I can identify when it is in the 'correct' state and when it has switched. That should also make it easier to threshold the derivative to identify a switch.